In [1]:
import re
import numpy as np 
import pandas as pd
from PIL import Image
from os import path, getcwd

import tweepy 
from textblob import TextBlob
import nltk

from wordcloud import WordCloud, ImageColorGenerator, STOPWORDS
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Scrape Twitter for #100DaysOfMlCode into DataFrame

In [2]:
CONSUMER_KEY = "your_key"
CONSUMER_SECERET = 'your_secret'
ACCESS_TOKEN = "your_key"
ACCESS_TOKEN_SECRET = "your_secret"'

In [3]:
def twitter_setup():
    """
    Twitter's API setup function.
    """
    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth)
    return api

In [4]:
extractor = twitter_setup()
tweets=[]

for tweet in tweepy.Cursor(extractor.search,q="100DaysOfMLCode",count=200,
                           lang="en").items():tweets.append(tweet.text)

print("Number of tweets extracted: {}.\n".format(len(tweet.text)))

Number of tweets extracted: 131.



In [5]:
str1 = ''.join(tweets)
data = pd.DataFrame(data=[tweet for tweet in tweets], columns=['Tweets'])
data.to_csv('data/100DOMC.csv')
data.head()


,Tweets
0,#100daysOfMLCode D72 Continued training mask R...
1,@Fluid65869181 @100DaysOfMLCode If you're inte...
2,RT @Grananqvist: #100DaysOfMLCode Day 70 - Sat...
3,#100DaysOfMLCode Day 70 - Sat on a plane for 1...


## Clean and Check Sentiment

In [6]:
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def analize_sentiment(tweet):
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else:
        return -1
    
    
data['SA'] = np.array([ analize_sentiment(tweet) for tweet in data['Tweets'] ])
data.head()

,Tweets,SA
0,#100daysOfMLCode D72 Continued training mask R...,0
1,@Fluid65869181 @100DaysOfMLCode If you're inte...,1
2,RT @Grananqvist: #100DaysOfMLCode Day 70 - Sat...,0
3,#100DaysOfMLCode Day 70 - Sat on a plane for 1...,0


In [7]:
pos_tweets = [ tweet for index, tweet in enumerate(data['Tweets']) if data['SA'][index] > 0]
neu_tweets = [ tweet for index, tweet in enumerate(data['Tweets']) if data['SA'][index] == 0]
neg_tweets = [ tweet for index, tweet in enumerate(data['Tweets']) if data['SA'][index] < 0]

print("Percentage of positive tweets: {}%".format(len(pos_tweets)*100/len(data['Tweets'])))
print("Percentage of neutral tweets: {}%".format(len(neu_tweets)*100/len(data['Tweets'])))
print("Percentage de negative tweets: {}%".format(len(neg_tweets)*100/len(data['Tweets'])))

Percentage of positive tweets: 25.0%
Percentage of neutral tweets: 75.0%
Percentage de negative tweets: 0.0%


## Process Tweets for WordCloud Visualization

In [8]:
def processTweet2(str1):
    str1 = str1.lower()
    str1 = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',str1)
    str1 = re.sub('@[^\s]+...','',str1)
    str1 = re.sub('[\s]+', ' ', str1)
    str1 = re.sub(r'#([^\s]+)', r'\1', str1)
    str1 = str1.strip('\'"')
    return str1    

In [9]:
str2= processTweet2(str1)
blob= TextBlob(str2)
str3=[]
count=0
for np in blob.noun_phrases:
  
  str3.append(np)
  count+=1
str4=''.join(str3)

In [ ]:
#nltk.download('brown')

d = getcwd()

text= ''.join(str2)
stp=list(set(STOPWORDS))
w=['hi','ur','dear','rt','even','really','still','...','now','please','getting','guy','want','day','will','using','make','give']
q=w+stp


mask = np.array(Image.open(path.join(d, ""image/Twitter-Logo.jpg"")))
wc = WordCloud(background_color="white", max_words=500, mask=mask,stopwords=q,max_font_size=90, random_state=42)
wc.generate(text)
image_colors = ImageColorGenerator(mask)
plt.figure(figsize=[10,10],dpi=400)
plt.tight_layout(pad=0)
plt.imshow(wc.recolor(color_func=image_colors), interpolation="bilinear")
plt.axis("off")
plt.show()